In [1]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import statistics
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Short-term forecasting

Key influences
* Market sentiment
* Technical indicators like moving averages, RSI, MACD
* Order flow
* Volatility

Features
* Prices
* Trading volume
* Bid-ask spreads
* News sentiment - Can incorperate later

In [2]:
df = yf.Ticker('AMZN').history(start='2020-01-01', end=date.today())

# contains historical stock prices, volume, dividends
history = df

# open and close are the closest free & available approximation of the bid-ask spread
history['Bid-Ask Spread'] = abs(df['Close'] - df['Open'])
history = history[['Volume', 'Bid-Ask Spread', 'Volume', 'Close']]

In [3]:
# calculating RSI (relative strength indicator)
def RSI(df, window=14):
  delta = df['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = abs(delta.where(delta < 0, 0))

  avg_gain = gain.rolling(window=window, min_periods=1).mean()
  avg_loss = loss.rolling(window=window, min_periods=1).mean()

  RS = avg_gain / avg_loss
  return 100 - (100/(1+RS))

# creating MACD divergence indicator
def MACD_ind(df):
  close = df['Close']

  ema_12 = close.ewm(span=12, adjust=False).mean()
  ema_26 = close.ewm(span=26, adjust=False).mean()

  macd = ema_12 - ema_26
  signal_line = macd.ewm(span=9, adjust=False).mean()

  return macd < signal_line

In [4]:
history['RSI'] = RSI(history)
history['MACD Diverges'] = MACD_ind(history)

<ipython-input-4-e6d6912698a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['RSI'] = RSI(history)
<ipython-input-4-e6d6912698a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  history['MACD Diverges'] = MACD_ind(history)


In [5]:
history.head()

,Volume,Bid-Ask Spread,Volume,Close,RSI,MACD Diverges
Date,,,,,,
2020-01-02 00:00:00-05:00,80580000,1.150497,80580000,94.900497,NaN,False
2020-01-03 00:00:00-05:00,75288000,0.523499,75288000,93.748497,0.000000,True
2020-01-06 00:00:00-05:00,81236000,2.143997,81236000,95.143997,54.779189,True
2020-01-07 00:00:00-05:00,80898000,0.118004,80898000,95.343002,58.055779,False
2020-01-08 00:00:00-05:00,70160000,0.303497,70160000,94.598503,45.674679,True


In [ ]:
# to do

# calculate IV (implied volitality) - for market sentiment
# get news sentiment